In [161]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

In [ ]:
# Scrape lounge data from United website
url = 'https://www.united.com/en/us/fly/travel/airport/united-club-and-lounge-locations.html'

driver = webdriver.Chrome()
driver.get(url)

wait = WebDriverWait(driver, 10)

dropdown = Select(wait.until(EC.presence_of_element_located((By.ID, "lookup-airport-united-lounges"))))
rows_all = []

for option in dropdown.options[1:]:
    value = option.get_attribute('value')
    print('Selecting:', value.upper())

    dropdown.select_by_value(value)

    time.sleep(2)
    
    tables = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//table")))
    for i, table in enumerate(tables):
        rows = table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            rows_all.append([value.upper(), i+1]+[col.text for col in cols])

Selecting: ATL
--- Table 1 ---
Selecting: AUS
--- Table 1 ---
Selecting: bos
--- Table 1 ---
Selecting: ORD
--- Table 1 ---
--- Table 2 ---
Selecting: CLE
--- Table 1 ---
Selecting: DFW
--- Table 1 ---
Selecting: DEN
--- Table 1 ---
--- Table 2 ---
Selecting: FLL
--- Table 1 ---
Selecting: GUM
--- Table 1 ---
Selecting: HKG
--- Table 1 ---
Selecting: HNL
--- Table 1 ---
Selecting: IAH
--- Table 1 ---
--- Table 2 ---
--- Table 3 ---
Selecting: LAS
--- Table 1 ---
Selecting: LAX
--- Table 1 ---
--- Table 2 ---
Selecting: LHR
--- Table 1 ---
Selecting: MEX
--- Table 1 ---
Selecting: MSP
--- Table 1 ---
Selecting: MSY
--- Table 1 ---
Selecting: EWR
--- Table 1 ---
--- Table 2 ---
Selecting: LGA
--- Table 1 ---
Selecting: SNA
--- Table 1 ---
Selecting: MCO
--- Table 1 ---
Selecting: PDX
--- Table 1 ---
Selecting: PHL
--- Table 1 ---
Selecting: PHX
--- Table 1 ---
Selecting: RDU
--- Table 1 ---
Selecting: SAT
--- Table 1 ---
Selecting: SAN
--- Table 1 ---
Selecting: SFO
--- Table 1 ---
--- T

In [234]:
# Combine lounge data into df
cols = ['airport', 'type', 'location', 'hours', 'amenities']

df = pd.DataFrame(rows_all, columns=cols)
df = df[df['amenities'].notna()]

In [ ]:
# Clean lounge hours
df['hours'] = df['hours'].str.split('\n')
df = df.explode('hours')

df['days'] = df['hours'].str.replace(r"\d|:|-|–|a\.m\.|p\.m\.|p\.m", "", regex=True)
df_days = df['days'].str.strip().str.lower().str.replace(', ', '|').str.replace(' and ', '|', regex=True).str.replace('\s+', '_', regex=True).str.get_dummies(sep='|')
df_days.loc[df_days['daily']==1, ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']] = 1
df_days.loc[df_days['sun_fri']==1, ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'sunday']] = 1

df_hours = df['hours'].str.replace(r"\.", "", regex=True).str.findall(r"\d|:|-|–|am|pm").str.join('').str.split('[-–]', expand=True)[[0, 1]]
df_hours.columns = ['open', 'close']

In [ ]:
df_amenities = df['amenities'].str.replace('\n', '|').str.replace('[\s+-]', '_', regex=True).str.lower().str.get_dummies(sep='|')
